In [1]:
import pandas as pd
import numpy as np
import tensorflow

In [2]:
ls

 Volume in drive C is Acer
 Volume Serial Number is 1023-C329

 Directory of C:\Users\Uttam\Desktop\test_env\seq_generator

05-08-2020  17:05    <DIR>          .
05-08-2020  17:05    <DIR>          ..
05-08-2020  15:42    <DIR>          .ipynb_checkpoints
05-08-2020  17:02           122,420 Untitled.ipynb
05-08-2020  15:40           167,518 wonderland.txt
               2 File(s)        289,938 bytes
               3 Dir(s)  91,945,644,032 bytes free


In [3]:
#importing libraries
import numpy as np
import pandas as pd
import sys 
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Activation, Flatten, Dropout, Dense, Embedding, TimeDistributed
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical

In [4]:

# load ascii text and covert to lowercase
filename = "wonderland.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

In [5]:
raw_text[:105]

"\ufeffproject gutenberg's alice's adventures in wonderland, by lewis carroll\n\nthis ebook is for the use of any"

In [6]:
set(raw_text)

{'\n',
 ' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 "'",
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 '@',
 '[',
 ']',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '\ufeff'}

In [7]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [8]:
char_to_int

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '#': 4,
 '$': 5,
 '%': 6,
 "'": 7,
 '(': 8,
 ')': 9,
 '*': 10,
 ',': 11,
 '-': 12,
 '.': 13,
 '/': 14,
 '0': 15,
 '1': 16,
 '2': 17,
 '3': 18,
 '4': 19,
 '5': 20,
 '6': 21,
 '7': 22,
 '8': 23,
 '9': 24,
 ':': 25,
 ';': 26,
 '?': 27,
 '@': 28,
 '[': 29,
 ']': 30,
 '_': 31,
 'a': 32,
 'b': 33,
 'c': 34,
 'd': 35,
 'e': 36,
 'f': 37,
 'g': 38,
 'h': 39,
 'i': 40,
 'j': 41,
 'k': 42,
 'l': 43,
 'm': 44,
 'n': 45,
 'o': 46,
 'p': 47,
 'q': 48,
 'r': 49,
 's': 50,
 't': 51,
 'u': 52,
 'v': 53,
 'w': 54,
 'x': 55,
 'y': 56,
 'z': 57,
 '\ufeff': 58}

In [9]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  163781
Total Vocab:  59


In [10]:
raw_text[0:101], raw_text[101]

("\ufeffproject gutenberg's alice's adventures in wonderland, by lewis carroll\n\nthis ebook is for the use of",
 ' ')

In [11]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  163681


In [12]:
dataX[:50]

[[58,
  47,
  49,
  46,
  41,
  36,
  34,
  51,
  1,
  38,
  52,
  51,
  36,
  45,
  33,
  36,
  49,
  38,
  7,
  50,
  1,
  32,
  43,
  40,
  34,
  36,
  7,
  50,
  1,
  32,
  35,
  53,
  36,
  45,
  51,
  52,
  49,
  36,
  50,
  1,
  40,
  45,
  1,
  54,
  46,
  45,
  35,
  36,
  49,
  43,
  32,
  45,
  35,
  11,
  1,
  33,
  56,
  1,
  43,
  36,
  54,
  40,
  50,
  1,
  34,
  32,
  49,
  49,
  46,
  43,
  43,
  0,
  0,
  51,
  39,
  40,
  50,
  1,
  36,
  33,
  46,
  46,
  42,
  1,
  40,
  50,
  1,
  37,
  46,
  49,
  1,
  51,
  39,
  36,
  1,
  52,
  50,
  36,
  1,
  46],
 [47,
  49,
  46,
  41,
  36,
  34,
  51,
  1,
  38,
  52,
  51,
  36,
  45,
  33,
  36,
  49,
  38,
  7,
  50,
  1,
  32,
  43,
  40,
  34,
  36,
  7,
  50,
  1,
  32,
  35,
  53,
  36,
  45,
  51,
  52,
  49,
  36,
  50,
  1,
  40,
  45,
  1,
  54,
  46,
  45,
  35,
  36,
  49,
  43,
  32,
  45,
  35,
  11,
  1,
  33,
  56,
  1,
  43,
  36,
  54,
  40,
  50,
  1,
  34,
  32,
  49,
  49,
  46,
  43,
  43,
  0,
  

In [13]:
dataY[:5]

[37, 1, 32, 45, 56]


### 1. transform the list of input sequences into the form [samples, time steps, features] expected by an LSTM network.
   link : https://medium.com/@shivajbd/understanding-input-and-output-shape-in-lstm-keras-c501ee95c65e

### 2.  rescale the integers to the range 0-to-1 to make the patterns easier to learn by the LSTM network that uses the sigmoid activation function by default.

### 3. we need to convert the output patterns (single characters converted to integers) into a one hot encoding. This is so that we can configure the network to predict the probability of each of the 47 different characters in the vocabulary (an easier representation) rather than trying to force it to predict precisely the next character.Each y value is converted into a sparse vector with a length of 47, full of zeros except with a 1 in the column for the letter (integer) that the pattern represents

In [14]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
X

array([[[58],
        [47],
        [49],
        ...,
        [36],
        [ 1],
        [46]],

       [[47],
        [49],
        [46],
        ...,
        [ 1],
        [46],
        [37]],

       [[49],
        [46],
        [41],
        ...,
        [46],
        [37],
        [ 1]],

       ...,

       [[51],
        [46],
        [ 1],
        ...,
        [46],
        [46],
        [42]],

       [[46],
        [ 1],
        [39],
        ...,
        [46],
        [42],
        [50]],

       [[ 1],
        [39],
        [36],
        ...,
        [42],
        [50],
        [13]]])

In [15]:
X.shape

(163681, 100, 1)

In [16]:
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = to_categorical(dataY)

In [17]:
X

array([[[0.98305085],
        [0.79661017],
        [0.83050847],
        ...,
        [0.61016949],
        [0.01694915],
        [0.77966102]],

       [[0.79661017],
        [0.83050847],
        [0.77966102],
        ...,
        [0.01694915],
        [0.77966102],
        [0.62711864]],

       [[0.83050847],
        [0.77966102],
        [0.69491525],
        ...,
        [0.77966102],
        [0.62711864],
        [0.01694915]],

       ...,

       [[0.86440678],
        [0.77966102],
        [0.01694915],
        ...,
        [0.77966102],
        [0.77966102],
        [0.71186441]],

       [[0.77966102],
        [0.01694915],
        [0.66101695],
        ...,
        [0.77966102],
        [0.71186441],
        [0.84745763]],

       [[0.01694915],
        [0.66101695],
        [0.61016949],
        ...,
        [0.71186441],
        [0.84745763],
        [0.22033898]]])

In [18]:
y.shape

(163681, 58)

In [19]:
X.shape

(163681, 100, 1)

In [20]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

. The output layer is a Dense layer using the softmax activation function to output a probability prediction for each of the 47 characters between 0 and 1.

In [21]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 256)               264192    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 58)                14906     
Total params: 279,098
Trainable params: 279,098
Non-trainable params: 0
_________________________________________________________________


In [23]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [24]:
model.fit(X, y, epochs=50, batch_size=128, callbacks=callbacks_list)

Train on 163681 samples
Epoch 1/50
163456/163681 [============================>.] - ETA: 0s - loss: 2.9775- ETA: 0s - loss:  - ETA: 0s - loss: 2
Epoch 00001: loss improved from inf to 2.97722, saving model to weights-improvement-01-2.9772.hdf5
163681/163681 [==============================] - 53s 324us/sample - loss: 2.9772
Epoch 2/50
163456/163681 [============================>.] - ETA: 0s - loss: 2.8047
Epoch 00002: loss improved from 2.97722 to 2.80466, saving model to weights-improvement-02-2.8047.hdf5
163681/163681 [==============================] - 48s 290us/sample - loss: 2.8047
Epoch 3/50
163584/163681 [============================>.] - ETA: 0s - loss: 2.7158
Epoch 00003: loss improved from 2.80466 to 2.71584, saving model to weights-improvement-03-2.7158.hdf5
163681/163681 [==============================] - 44s 268us/sample - loss: 2.7158
Epoch 4/50
163456/163681 [============================>.] - ETA: 0s - loss: 2.6430
Epoch 00004: loss improved from 2.71584 to 2.64296, saving

In [25]:
 ls

 Volume in drive C is Acer
 Volume Serial Number is 1023-C329

 Directory of C:\Users\Uttam\Desktop\test_env\seq_generator

05-08-2020  17:33    <DIR>          .
05-08-2020  17:33    <DIR>          ..
05-08-2020  15:42    <DIR>          .ipynb_checkpoints
05-08-2020  17:33           123,576 Untitled.ipynb
05-08-2020  17:07         3,377,488 weights-improvement-01-2.9772.hdf5
05-08-2020  17:08         3,377,488 weights-improvement-02-2.8047.hdf5
05-08-2020  17:08         3,377,488 weights-improvement-03-2.7158.hdf5
05-08-2020  17:09         3,377,488 weights-improvement-04-2.6430.hdf5
05-08-2020  17:10         3,377,488 weights-improvement-05-2.5867.hdf5
05-08-2020  17:11         3,377,488 weights-improvement-06-2.5337.hdf5
05-08-2020  17:11         3,377,488 weights-improvement-07-2.4822.hdf5
05-08-2020  17:12         3,377,488 weights-improvement-08-2.4342.hdf5
05-08-2020  17:13         3,377,488 weights-improvement-09-2.3904.hdf5
05-08-2020  17:13         3,377,488 weights-improvemen

###  Generating text with LSTM

In [27]:

# load the network weights
filename = "weights-improvement-49-1.6946.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [28]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [31]:
# pick a random seed
start = np.random.randint(0, len(dataX)-1) # generating random number between o to len(dataX)
print(start)
pattern = dataX[start] #getting values od data_X on that random generated number
print(pattern)
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"") # getting number to char using dictionary

9927
[52, 43, 35, 1, 45, 46, 51, 1, 49, 36, 44, 36, 44, 33, 36, 49, 0, 36, 53, 36, 49, 1, 39, 32, 53, 40, 45, 38, 1, 50, 36, 36, 45, 1, 50, 52, 34, 39, 1, 32, 1, 51, 39, 40, 45, 38, 13, 0, 0, 32, 37, 51, 36, 49, 1, 32, 1, 54, 39, 40, 43, 36, 11, 1, 37, 40, 45, 35, 40, 45, 38, 1, 51, 39, 32, 51, 1, 45, 46, 51, 39, 40, 45, 38, 1, 44, 46, 49, 36, 1, 39, 32, 47, 47, 36, 45, 36, 35, 11, 1]
Seed:
" uld not remember
ever having seen such a thing.

after a while, finding that nothing more happened,  "


In [32]:
# generate characters
for i in range(1000):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("Done")

and the thrt hele that sheel toentidg to toeek that she was not of sieht arainer. 
'i woshe you dave tasee tutnone!' she gatter seilied in a sonee of beutarer. 
'thet's no tuerents to seaee ' said the caterpillar.

'well, it soue thing wase'  she daterpirlar nanliry an the courors as hh sooke. 
'whu don't know it,' said the caterpillar.

'well, it soat doneg tt heve,' said the kacci hartily, 'so they ware to an tneec tf deineve them tomntenn. and then so teee thet so be in on dinn them some time wouh the oeee tf thisg theee thee so the kohe turtle so herself the had toeerid the was not of the sabds, and she west oooe toinein th the carer tr seye the rome of the table, aut the waite ras thr op hit tore oo lete to beanns, and she whstght the was not in the was wo ce aound to the thyee saren of the courd, and was going to tie kacts wo tei that sam the had toe the same was so metee tu the wable, she was toe riilliin that she was not an angos tote of the carke and the carter, and the thitgh